# Tuning Deep Feature Synthesis

There are several parameters that can be tuned to change the output of DFS. We'll explore these parameters using the following `transactions` EntitySet.

In [ ]:
import featuretools as ft

es = ft.demo.load_mock_customer(return_entityset=True)
es

## Using "Seed Features"

Seed features are manually defined and problem specific features that a user provides to DFS. Deep Feature Synthesis will then automatically stack new features on top of these features when it can.

By using seed features, we can include domain specific knowledge in feature engineering automation. For the seed feature below, the domain knowlege may be that, for a specific retailer, a transaction above $125 would be considered an expensive purchase.

In [ ]:
expensive_purchase = ft.Feature(es["transactions"].ww["amount"]) > 125

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["percent_true"],
    seed_features=[expensive_purchase],
)
feature_matrix[["PERCENT_TRUE(transactions.amount > 125)"]]

We can now see that the ``PERCENT_TRUE`` primitive was automatically applied to the boolean `expensive_purchase` feature from the `transactions` table. The feature produced as a result can be understood as the percentage of transactions for a customer that are considered expensive.

## Add "interesting" values to columns

Sometimes we want to create features that are conditioned on a second value before calculations are performed. We call this extra filter a "where clause". Where clauses are used in Deep Feature Synthesis by including primitives in the `where_primitives` parameter to DFS.

By default, where clauses are built using the ``interesting_values`` of a column.

Interesting values can be automatically determined and added for each DataFrame in a pandas EntitySet by calling `es.add_interesting_values()`.

In [ ]:
values_dict = {"device": ["desktop", "mobile", "tablet"]}
es.add_interesting_values(dataframe_name="sessions", values=values_dict)

Interesting values are stored in the DataFrame's Woodwork typing information.

In [ ]:
es["sessions"].ww.columns["device"].metadata

Now that interesting values are set for the `device` column in the `sessions` table, we can specify the aggregation primitives for which we want where clauses using the ``where_primitives`` parameter to DFS.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["count", "avg_time_between"],
    where_primitives=["count", "avg_time_between"],
    trans_primitives=[],
)
feature_matrix

Now, we have several new potentially useful features. Here are two of them that are built off of the where clause "where the device used was a tablet":

In [ ]:
feature_matrix[
    [
        "COUNT(sessions WHERE device = tablet)",
        "AVG_TIME_BETWEEN(sessions.session_start WHERE device = tablet)",
    ]
]

The first geature, `COUNT(sessions WHERE device = tablet)`, can be understood as indicating *how many sessions a customer completed on a tablet*.

The second feature, `AVG_TIME_BETWEEN(sessions.session_start WHERE device = tablet)`, calculates *the time between those sessions*.

We can see that customer who only had 0 or 1 sessions on a tablet had ``NaN`` values for average time between such sessions.


## Encoding categorical features

Machine learning algorithms typically expect all numeric data or data that has defined numeric representations, like boolean values corresponding to `0` and `1`. When Deep Feature Synthesis generates categorical features, we can encode them using Featureools.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["mode"],
    trans_primitives=["time_since"],
    max_depth=1,
)

feature_matrix

This feature matrix contains 2 columns that are categorical in nature, ``zip_code`` and ``MODE(sessions.device)``. We can use the feature matrix and feature definitions to encode these categorical values into boolean values. Featuretools offers functionality to apply one hot encoding to the output of DFS.

In [ ]:
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)
feature_matrix_enc

The returned feature matrix is now encoded in a way that is interpretable to machine learning algorithms. Notice how the columns that did not need encoding are still included. Additionally, we get a new set of feature definitions that contain the encoded values.

In [ ]:
features_enc

These features can be used to calculate the same encoded values on new data. For more information on feature engineering in production, read the [Deployment](deployment.ipynb) guide.